In [4]:
import cv2


# 读入图像
img = cv2.imread('1.jpg')

# 转换为灰度图像
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 使用Otsu's方法进行二值化
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)


# 定义一个5x5的结构元素
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

# 进行开运算
opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)


In [2]:
import cv2
import pytesseract
from pytesseract import Output

# 读入图像
img = cv2.imread('1.jpg')

# 使用 Tesseract 进行文本检测，返回结果的数据结构为字典
d = pytesseract.image_to_data(img, output_type=Output.DICT)

# 遍历所有检测到的文本区域
n_boxes = len(d['text'])
for i in range(n_boxes):
    # 如果文本区域的置信度不为0
    if int(d['conf'][i]) > 0:
        # 获取文本区域的坐标
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])

        # 在原图上画出文本区域
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# 显示带有文本区域的图像
cv2.imshow('img', img)
cv2.waitKey(0)


ImportError: DLL load failed: 找不到指定的模块。

In [1]:
!git clone https://github.com/clovaai/CRAFT-pytorch.git


Cloning into 'CRAFT-pytorch'...


In [2]:
%cd CRAFT-pytorch


e:\softbei\code\CRAFT-pytorch


In [4]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch==0.4.1.post2 (from versions: 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1)
ERROR: No matching distribution found for torch==0.4.1.post2


In [1]:
!python test.py --trained_model=[path_to_your_weight_file] --test_folder=[path_to_your_images]


python: can't open file 'test.py': [Errno 2] No such file or directory


In [ ]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('1.jpg')

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Use Otsu's method to binarize the image
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Threshold for empty space
vertical_empty_space_threshold = binary.shape[1] * 255 * 0.02  # Adjust this value according to your image
horizontal_empty_space_threshold = binary.shape[0] * 255 * 0.02  # Adjust this value according to your image

# Initialize the previous split point
prev_vertical_split_point = 0

# Compute the vertical projection of the binary image
vertical_projection = np.sum(binary, axis=1)

# Find the empty spaces
vertical_empty_spaces = np.where(vertical_projection <= vertical_empty_space_threshold)[0]

# Initialize the list of split points
vertical_split_points = []

# Initialize the previous empty space
prev_vertical_empty_space = -1

# Loop over the empty spaces
for empty_space in vertical_empty_spaces:
    if prev_vertical_empty_space != -1 and empty_space - prev_vertical_empty_space > 1:
        # If there is a gap between two empty spaces, add a split point
        vertical_split_points.append((prev_vertical_empty_space + empty_space) // 2)
    prev_vertical_empty_space = empty_space

# Add the end of the image as the last split point
vertical_split_points.append(binary.shape[0])

# Loop over the split points
for i, vertical_split_point in enumerate(vertical_split_points):
    # Cut the image at the split point
    vertical_part = img[prev_vertical_split_point:vertical_split_point, :]

    # Compute the horizontal projection of the binary image
    horizontal_projection = np.sum(binary[prev_vertical_split_point:vertical_split_point, :], axis=0)

    # Find the empty spaces
    horizontal_empty_spaces = np.where(horizontal_projection <= horizontal_empty_space_threshold)[0]

    # Initialize the list of split points
    horizontal_split_points = []

    # Initialize the previous empty space
    prev_horizontal_empty_space = -1

    # Loop over the empty spaces
    for empty_space in horizontal_empty_spaces:
        if prev_horizontal_empty_space != -1 and empty_space - prev_horizontal_empty_space > 1:
            # If there is a gap between two empty spaces, add a split point
            horizontal_split_points.append((prev_horizontal_empty_space + empty_space) // 2)
        prev_horizontal_empty_space = empty_space

    # Add the end of the image as the last split point
    horizontal_split_points.append(vertical_part.shape[1])

    # Initialize the previous split point
    prev_horizontal_split_point = 0

    # Loop over the split points
    for j, horizontal_split_point in enumerate(horizontal_split_points):
        # Cut the image at the split point
        part = vertical_part[:, prev_horizontal_split_point:horizontal_split_point]

        # Save the part
        cv2.imwrite(f'picture/part_{i}_{j}.jpg', part)

        # Update the previous split point
        prev_horizontal_split_point = horizontal_split_point

    # Update the previous split point
    prev_vertical_split_point = vertical_split_point
